In [1]:
# _*_ coding: UTF-8 _*_
#coding:utf-8
from urllib.request import urlopen
from selenium import webdriver
from bs4 import BeautifulSoup as bs
import pandas as pd
import csv
from selenium.webdriver.remote.webelement import WebElement
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
import re
import numpy as np
import openpyxl

In [2]:
#매크로용 웹켜기
#driver = webdriver.Remote('http://localhost:4444/wd/hub', options.to_capabilities())  # 셀레 서버를 Standalone 방식으로 가동
driver = webdriver.Chrome('./chromedriver')
driver.implicitly_wait(5)

SessionNotCreatedException: Message: session not created: This version of ChromeDriver only supports Chrome version 75


In [5]:
href_list = []   
for n in range(1,3):
    try:
        base_url = f'https://www.latimes.com/search?q=business&p={n}'
        driver.get(base_url)
        soup = bs(driver.page_source, 'html.parser')
        dt = soup.find_all('div', {'class':'PromoMedium-title'}) # 링크 선별 작업
        dt1 = re.findall('href\=\"(.*?)\"\>', str(dt)) # 선별된 div 중에 링크부분만 가져오는 정규 표현식
        href = dt1[0::2] # 홀수만 가져오기, 짝수가져와도 상관 없음
        href_list.extend(href)
    except Exception as e:      
        print(e)
        continue

In [4]:
#링크 저장
df1 = pd.DataFrame(href_list)
df1.to_excel('latimes_href2.xlsx', sheet_name = 'sheet1')

In [14]:
res_list = []
for href in href_list:
    content_total = []
    try:
        html = urlopen(href)
        soup = bs(html, "html.parser")
        title = soup.find("h1", {"ArticlePage-headline"}).get_text()
        title = title.strip().replace('\xa0','')
        content_total.extend([ p.get_text() for p in soup.find_all('p')])
        content_total = content_total[0:-7]
        content = ' '.join(content_total)
        content = content.replace('\xa0','')
    except Exception as e:
        print(e)
        continue
    res_list.append({'title' : title, 'content': content})

'NoneType' object has no attribute 'get_text'


In [15]:
res_list

[{'title': 'Nearly 3,000 illegal marijuana businesses found in California audit, dwarfing legal trade',
  'content': 'California’s black market for cannabis is at least three times the size of its regulated weed industry, according to an audit made public Wednesday, the latest indication of the state’s continued struggle to tame a cannabis economy that has long operated in legal limbo. The audit, conducted by the United Cannabis Business Assn., found approximately 2,835 unlicensed dispensaries and delivery services operating in California. By comparison, only 873 cannabis sellers in the state are licensed, according to the Bureau of Cannabis Control. The figures are the latest sign of California’s rocky rollout of its legal marketplace, which promised better regulations and control beginning in 2018. Legitimate marijuana businesses have repeatedly criticized state leaders and law enforcement for failing to curb unlicensed dispensaries and delivery services, which sell cannabis at a muc

In [21]:
content_total.append([ i.get_attribute('p') for i in driver.find_elements_by_css_selector('div.board-list > div.inner_list > a.article')])

AttributeError: ResultSet object has no attribute 'p'. You're probably treating a list of items like a single item. Did you call find_all() when you meant to call find()?

In [52]:
res_list = []
for article in href_list:
        driver.get(article)
        soup = bs(driver.page_source, 'html.parser')
        title = soup.find("h1", {"ArticlePage-headline"}).get_text()
        content = soup.find_all("p").get_text()
        try:
            content = soup.find("div", {"id":"tbody"}).get_text()
            content = re.split('[0-9]\.', content)
            nickname = content[1]
            age = content[2]
            age = age[9:-1]
            address = content[3]
            address = address[6:]
            vehi = content[4]
            vehi = vehi[10:]
        except Exception as e:
            print(e)
            continue
        res_list.append({'title' : title, 'user_id': user_id, 'nickname' : nickname, 'age' : age, 'address' : address, 'vehi' : vehi})

['https://www.latimes.com/california/story/2019-08-16/2-shot-at-busy-north-hollywood-intersection',
 'https://www.latimes.com/california/story/2019-09-04/california-conception-dive-boat-fire-victims',
 'https://www.latimes.com/business/story/2019-08-19/in-shocking-reversal-big-business-puts-the-shareholder-value-myth-in-the-grave',
 'https://www.latimes.com/california/story/2019-07-24/broguieres-dairy-montebello-closed',
 'https://www.latimes.com/sports/story/2019-08-17/sports-report',
 'https://www.latimes.com/california/story/2019-09-10/national-city-enacts-ban-on-retail-sales-of-dogs-cats-and-rabbits',
 'https://www.latimes.com/california/story/2019-09-10/santa-monica-schools-lockdown-police-search-robbery-suspect',
 'https://www.latimes.com/california/story/2019-07-18/video-serial-robbery-suspect-los-angeles-inglewood-reward',
 'https://www.latimes.com/california/story/2019-09-09/la-college-admissions-case-sentences-how-much-parents-paid',
 'https://www.latimes.com/entertainment-ar

In [113]:
#특정 페이지부터 실행 코드
href_list4 = []   #print(href.get('href'))
for n in range(46,83):  # 83까지해야함
    if n < 81:
        driver.find_element_by_xpath('//*[@id="searchresults"]/div/div[3]/ul/li[9]/a').click()
        soup = bs(driver.page_source, 'html.parser')
        href_list4.extend([ i.get('href') for i in soup.find_all("a", {"ng-bind-html":"org.OrganizationName | decodeUTF8"})])
    elif n >= 81:
        driver.find_element_by_xpath(f'//*[@id="searchresults"]/div/div[3]/ul/li[{n-71}]/a').click()
        soup = bs(driver.page_source, 'html.parser')
        href_list.extend([ i.get('href') for i in soup.find_all("a", {"ng-bind-html":"org.OrganizationName | decodeUTF8"})])

In [120]:
href_list5 = href_list4[88:]

In [117]:
df1 = pd.DataFrame(href_list4)
df1.to_excel('urllist_ufl4.xlsx', sheet_name = 'sheet1')

In [ ]:
#for article in href_list:
 #       driver.get('https://orgs.studentinvolvement.ufl.edu' + article)
#
 #           title = soup1.find("h1", {"style":"padding: 13px 0px 0px 85px;"}).get_text()
 #           soup2 = soup1.find("div", {"style":"margin-left: 5px; padding: 5px 15px; border-left: 1px solid rgb(210, 210, 210);"})
 #           email = soup2.select('div')[-1].get_text()[19:-1]
  #      except Exception as e:
   #         print(e)
    #        continue
     # res_list.append({'title' : title, 'email' : email})